In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece

In [50]:
## 学習済t5モデルのダウンロード
!cd "drive/My Drive/ProfileGenerator/";wget https://www.dropbox.com/s/zpi7wltm8v5oo6c/t5.tar.gz
!cd "drive/My Drive/ProfileGenerator/";tar xvfz "t5.tar.gz"

--2021-12-11 10:13:27--  https://www.dropbox.com/s/zpi7wltm8v5oo6c/t5.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zpi7wltm8v5oo6c/t5.tar.gz [following]
--2021-12-11 10:13:27--  https://www.dropbox.com/s/raw/zpi7wltm8v5oo6c/t5.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc81d1fbbb4bdd6482329029ba87.dl.dropboxusercontent.com/cd/0/inline/Bbp7dgNtNZxwNz7qRPLx7gydq6knNEskJdHBFrtUMLs9ndSvi-ebi7NLEnAbW8RgRPU6rEA3NrBx0MvfO4bWzsKMyeZbB0Pwg94xldntI7vRFb58G3goUYtbGns8iE4_RxF6hqvlQYPr8Hi6mLRlkbyD/file# [following]
--2021-12-11 10:13:28--  https://uc81d1fbbb4bdd6482329029ba87.dl.dropboxusercontent.com/cd/0/inline/Bbp7dgNtNZxwNz7qRPLx7gydq6knNEskJdHBFrtUMLs9ndSvi-ebi7NLEnAbW8RgRPU6rEA3NrBx0MvfO4bWzsKMyeZbB0Pwg9

In [5]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
import codecs
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

main_path = "/content/drive/My Drive/ProfileGenerator/"

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [9]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,


    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  30,  # 入力文の最大トークン数
    "max_target_length": 512,  # 出力文の最大トークン数
    "train_batch_size":  8,  # 訓練時のバッチサイズ
    "eval_batch_size":   8,  # テスト時のバッチサイズ
    "num_train_epochs":  8,  # 訓練するエポック数
    })

In [14]:
class ProfileDataset(Dataset):
    def __init__(self, tokenizer, list_input, input_max_len=512, target_max_len=512):
        self.list_input = list_input
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        source_mask = self.inputs[index]["attention_mask"].squeeze()
        return {"source_ids": source_ids, "source_mask": source_mask}
  
    def _build(self):
        for input in self.list_input:
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input], max_length=self.input_max_len, truncation=True, 
                padding="max_length", return_tensors="pt"
            )
            self.inputs.append(tokenized_inputs)

In [7]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

def get_output(args_dict,tokenizer,trained_model,list_input):
   # テストデータの読み込み
    test_dataset = ProfileDataset(tokenizer, list_input, 
                            input_max_len=args_dict["max_input_length"], 
                            target_max_len=args_dict["max_target_length"])

    test_loader = DataLoader(test_dataset, batch_size=8, num_workers=4)

    trained_model.eval()

    inputs = []
    outputs = []

    for batch in tqdm(test_loader):
        input_ids = batch['source_ids']
        input_mask = batch['source_mask']
        if USE_GPU:
            input_ids = input_ids.cuda()
            input_mask = input_mask.cuda()

        output = trained_model.generate(input_ids=input_ids, 
          attention_mask=input_mask, 
          max_length=args_dict["max_target_length"],
          repetition_penalty=10.0,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
            )

        output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=False) 
                  for ids in output]
        input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=False) 
                  for ids in input_ids]

        inputs.extend(input_text)
        outputs.extend(output_text)
    return inputs,outputs

In [11]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデル
MODEL_DIR = f"{main_path}/t5"
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)
# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(MODEL_DIR)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

In [60]:
#サンプルテキストを読み込んで生成
file_path = f"{main_path}/sample_text/sample_fixed.txt"
list_input = codecs.open(file_path,encoding="utf-8")
for input, output in zip(*get_output(args_dict,tokenizer,trained_model,list_input)):
    print("title:     " + input)
    print("generated: " + output)
    print()

  0%|          | 0/3 [00:20<?, ?it/s]

title:     巫女 妹 三 中 失敗 行動 何 政治 アニメ 本 目の当たり グレイ
generated: 【人名】の巫女で、【人名】の妹。三姉妹の中で唯一失敗を許さない行動を取るが、何かと政治に疎い一面も持つ。アニメ第1作では本性を目の当たりにしたグレイモンによって倒される。

title:     妹 夫 巫女 嘘 経験 教師 父 武装 社員 宇宙 2 母
generated: 【人名】の妹で、【人名】の夫。巫女である父に嘘をついていた経験があり、教師として働きたいと願っている。父が武装会社の社員だったこともあり、宇宙へ飛び立つことを夢見ている。第2期では母が行方不明になり、妹と共に暮らしている。

title:     ナンバー 国 妹 妻 巫女 体 公 時代 武器 免許 作 現在
generated: ナンバー2。【人名】国の皇女で、ユグドラシルの妹。妻は巫女であり、体に鎧を纏っている。公女時代は武器の免許を持っていなかったが、本作では現在の身分である。

title:     中 巫女 興味 妹 家 オーナー 精 サポート 幽霊 エル 手 クラスメート
generated: 中2。巫女に興味があり、妹の【人名】が住む家のオーナーを務める霊能者。精霊に精通しており、サポート役でもある。幽霊や妖怪を怖がるエルにも手を差し伸べている。クラスメートからは「お姉ちゃん」と呼ばれている。

title:     妹 巫女 会長 相手 為 2 種族 評議 権利 野
generated: 【人名】の妹で巫女。【人名】学園の生徒会長を務めており、その実力はクラスメイト相手にも引けを取らない程である為、2種族間の評議権争いでは野望を果たそうとしていた。

title:     巫女 漫画 抜 一 妹 正 復活 アニメ 二 訓練 授業 女の子
generated: 【人名】の巫女で、漫画版では抜一の妹。【人名】が正気を取り戻したことで復活する。アニメ第二期では訓練生として登場し、授業も担当している。女の子にモテるらしい。

title:     財閥 エネルギー 妹 巫女 試合 任務 生前 戦闘 服 見た目 オレンジ ドイツ
generated: 【人名】財閥の令嬢。エネルギーを操る能力を持つ。【人名】の妹で、巫女として試合に勝つことを任務としている。生前は戦闘服のよう

In [62]:
#サンプルテキストを読み込んで生成2
file_path = f"{main_path}/sample_text/sample_fixed.txt"
list_input = codecs.open(file_path,encoding="utf-8")
for input, output in zip(*get_output(args_dict,tokenizer,trained_model,list_input)):
    print("title:     " + input)
    print("generated: " + output)
    print()

  0%|          | 0/3 [00:00<?, ?it/s]

title:     巫女 妹 三 中 失敗 行動 何 政治 アニメ 本 目の当たり グレイ
generated: 【人名】の巫女で、【人名】の妹。三姉妹の中で唯一失敗を許さない行動を取るが、何かと政治に疎い一面も持つ。アニメ第1作では本性を目の当たりにしたグレイモンによって倒される。

title:     妹 夫 巫女 嘘 経験 教師 父 武装 社員 宇宙 2 母
generated: 【人名】の妹で、【人名】の夫。巫女である父に嘘をついていた経験があり、教師として働きたいと願っている。父が武装会社の社員だったこともあり、宇宙へ飛び立つことを夢見ている。第2期では母が行方不明になり、妹と共に暮らしている。

title:     ナンバー 国 妹 妻 巫女 体 公 時代 武器 免許 作 現在
generated: ナンバー2。【人名】国の皇女で、ユグドラシルの妹。妻は巫女であり、体に鎧を纏っている。公女時代は武器の免許を持っていなかったが、本作では現在の身分である。

title:     中 巫女 興味 妹 家 オーナー 精 サポート 幽霊 エル 手 クラスメート
generated: 中2。巫女に興味があり、妹の【人名】が住む家のオーナーを務める霊能者。精霊に精通しており、サポート役でもある。幽霊や妖怪を怖がるエルにも手を差し伸べている。クラスメートからは「お姉ちゃん」と呼ばれている。

title:     妹 巫女 会長 相手 為 2 種族 評議 権利 野
generated: 【人名】の妹で巫女。【人名】学園の生徒会長を務めており、その実力はクラスメイト相手にも引けを取らない程である為、2種族間の評議権争いでは野望を果たそうとしていた。

title:     巫女 漫画 抜 一 妹 正 復活 アニメ 二 訓練 授業 女の子
generated: 【人名】の巫女で、漫画版では抜一の妹。【人名】が正気を取り戻したことで復活する。アニメ第二期では訓練生として登場し、授業も担当している。女の子にモテるらしい。

title:     財閥 エネルギー 妹 巫女 試合 任務 生前 戦闘 服 見た目 オレンジ ドイツ
generated: 【人名】財閥の令嬢。エネルギーを操る能力を持つ。【人名】の妹で、巫女として試合に勝つことを任務としている。生前は戦闘服のよう

In [2]:
!pip install gensim==3.8.3

In [3]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel

def load_lda_and_dict():
    ldafile = f"{main_path}/lda/test.lda"
    dictfile = f"{main_path}/lda/test.dict"
    lda = LdaModel.load(ldafile)
    dictionary = corpora.Dictionary.load(dictfile)
    return lda,dictionary

def expand_zokusei(list_zokusei,lda,dictionary):
    bow = dictionary.doc2bow(list_zokusei)
    list_tupl = lda.get_document_topics(bow)
    list_term, list_prob = [], []
    for topicid, prob in list_tupl:
        list_term_tupl = lda.get_topic_terms(topicid,topn=100)
        list_term += [term for term, prob in list_term_tupl]
        list_prob += [prob for term, prob in list_term_tupl]

    get_term = list_zokusei+[word for word in set([dictionary[word_id] for word_id in random.choices(list_term, weights=list_prob, k=10)]) if word not in list_zokusei]
    first_half = get_term[:5]
    last_half = get_term[5:]
    random.shuffle(first_half)
    return first_half+last_half

def generate_zokusei(lda,dictionary):
    topics = lda.get_topics()
    topic_inds = random.choices([ri for ri in range(len(topics))],k=1)
    word_dist = [prob/len(topic_inds) for topic in topics for prob in topic]
    ind_dist = [ri for topic in topics for ri in range(len(topic))]
    ind_list = random.choices(ind_dist,weights= word_dist,k=10)
    list_word = [word for word in set([dictionary[ind] for ind in ind_list])]
    return list_word

In [15]:
#属性語を指定してプロフィール文を生成
lda,dictionary = load_lda_and_dict()
list_zokusei = "ツインテール 小学生".split(" ")
list_expanded_zokusei = [" ".join(expand_zokusei(list_zokusei,lda,dictionary)) for _ in range(20)]
for input, output in zip(*get_output(args_dict,tokenizer,trained_model,list_expanded_zokusei)):
    print("title:     " + input)
    print("generated: " + output)
    print()

  0%|          | 0/3 [00:00<?, ?it/s]

title:     弟 小学生 ツインテール 両親 好意 3 系 学園 姉 屋 世界
generated: 【人名】の弟で小学生。髪型はツインテール。両親に好意を抱いており、【人名】と3人でよくつるんでいる。体育会系の学園に通う姉の面倒見も良いが、やや恥ずかしがり屋な一面もある。世界中を旅しているらしい。

title:     代表 小学生 運動 2 ツインテール 兄 コンプレックス 部 もの 息子 部員
generated: 【人名】代表。小学生ながら運動神経抜群で、【人名】の2つ年上の兄にコンプレックスを抱いている。陸上部に所属していたものの、成績はあまり芳しくない模様。息子と同じく水泳部員である。

title:     組 部 ツインテール 小学生 風紀 姉 パン 話 部員 成績
generated: 1年組→2年C組の男子生徒。中等部3年生。ツインテールの小学生。風紀委員をしている姉がいる。パンが大好物で、第6話では女子部員として成績優秀であることが判明する。

title:     キャラ アニメ 小学生 1 ツインテール もの パン 新聞 息子 一 初代 卒業
generated: 【人名】のキャラ。アニメオリジナルキャラクター。小学生1年生。髪型はツインテール。好きなものはパンと新聞。嫌いなものは息子一人。初代最終回で【人名】が卒業式に着ていた服を譲り受けた。

title:     人物 小学生 彼女 ツインテール 2 ゲーム 所 長髪 性格 一 卒業
generated: 作中の人物。【人名】の小学生時代の同級生で、彼女のツインテールを2つにまとめている。ゲーム版では少し抜けた所がある長髪の女性だが、性格は大人びていておっとりしている。一度卒業したら卒業するつもりらしい。

title:     1 ツインテール 小学生 弟 好意 両親 住人 兄 救出 外見
generated: 1年。髪型はツインテール。小学生の弟がいる。【人名】に好意を抱いているが、両親や住人からは「兄ちゃん」と呼ばれている。妹救出作戦の最中、姉と瓜二つな外見で再会する。

title:     好意 キャラ 代表 ツインテール 小学生 高校 作 部 学園 姉 本人 双子
generated: 【人名】に好意を寄せているキャラの代表格。ツインテールの小学生で、高校1年生(

In [18]:
#ランダムに属性語を生成してプロフィール文を生成
lda,dictionary = load_lda_and_dict()
list_expanded_zokusei = [" ".join(generate_zokusei(lda,dictionary)) for _ in range(20)]
for input, output in zip(*get_output(args_dict,tokenizer,trained_model,list_expanded_zokusei)):
    print("title:     " + input)
    print("generated: " + output)
    print()

  0%|          | 0/3 [00:00<?, ?it/s]

title:     豪 験 親愛 人物 実 紫 少尉 媛 性格 式
generated: 豪胆で、験体に秀でている。【人名】を親愛する人物だが、実の姉である紫媛には頭が上がらない。性格は温厚かつおっとりとしており、式神や精霊にも優しく接している。

title:     気 キス 穢れ 試験 バンド 胸 愛 生徒 宇宙 毎回
generated: 気の弱い【人名】にキスをして穢れを祓わせる。試験中はヘアバンドで胸を隠している。愛する生徒には「宇宙」と呼ばせているが、毎回その度に嫌がっている。

title:     枢 その後 クラスメイト 友人 寿 少女 戦い 本人 ユーリ 3000
generated: 【人名】枢のクラスメイトで友人。【人名】寿に好意を寄せている少女。戦いを好まないが、本人曰く「ユーリは3000歳以上」らしい。

title:     年 際 部屋 動作 け 間柄 存在 安定 保険 者
generated: 【人名】が年老いた際、【人名】の住む部屋に引っ越してきた少女。動作は軽く「けっこうキツイ」という程度の間柄ではあるものの、存在自体は安定しており、「保険」として加入している者も多い。

title:     子供 研究 事 態度 親友 周 父 魔術 秋 接客
generated: 【人名】の子供時代の友人。【人名】が研究に没頭している事に対して、冷淡な態度で接する事が多い。親友である周の父から魔術を教わっており、秋からは「お兄ちゃん」と呼ばれている。接客業にも長けている。

title:     気 城 2 その後 能力 理解 もの 店主 先行 世界
generated: 【人名】の気を引こうと【人名】城に乗り込むが、2度目の襲撃で敗北する。その後能力は理解できたものの、店主に先行き不測の事態のため世界から追放される。

title:     飛鳥 上司 1 島 故郷 アニメ 抗争 プロ 歌唱 互
generated: 飛鳥の上司。第1話から登場。【人名】島を故郷としている。アニメでは「」と抗争するプロ歌手で、歌唱力も高い。互いを認め合う仲である。

title:     何 販売 重 敵 ゼロ ほど ミッキー 転送 相手 記念
generated: 何十年も販売員として働いており、重度の敵はゼロと言っていいほどいる。ミッキーの転送能力で